# Installation and Setup



```
# This is formatted as code
```

**Step 1 - Clone a copy of our project folder to the google collab**

In [ ]:
# Clone the project
!git clone https://github.com/jezsll/ict3104-team09-2023.git

Cloning into 'ict3104-team09-2023'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 232 (delta 3), reused 10 (delta 1), pack-reused 217
Receiving objects: 100% (232/232), 117.28 MiB | 16.11 MiB/s, done.
Resolving deltas: 100% (43/43), done.


**Step2 - Installing the necessary dependency and packages**

**Triton**
- Programming language for scalable GPU computing. Follow the [Triton Installation Guide](https://triton-lang.org/main/getting-started/installation.html).

**Hugging Face Packages**
- Diffusers: Tools for NLP model training and optimization. Learn more in the [official guide](https://huggingface.co/docs/diffusers/installation).
- Transformers: Powerful library for NLP. Explore features in the [Transformers documentation](https://huggingface.co/docs/transformers/index).
- Bitsandbytes: Collection of ML tools by Tim Dettmers. Details on [GitHub](https://github.com/TimDettmers/bitsandbytes).
- Imageio-ffmpeg: FFmpeg functionality for Imageio. Check the [GitHub repository](https://github.com/imageio/imageio-ffmpeg).
- Xformers: Transformer library by Facebook Research. Enhance with Diffusers for optimization. Explore on [GitHub](https://github.com/facebookresearch/xformers).
- Accelerate: Hugging Face's GPU/TPU training library. Discover on [GitHub](https://github.com/huggingface/accelerate).
- Omegaconf: Flexible configuration library. Documentation [here](https://omegaconf.readthedocs.io/en/2.3_branch/).
- Einops: Library for flexible tensor manipulation. Explore on [GitHub](https://github.com/arogozhnikov/einops).
- AV (PyAV): Multimedia library. Access the [GitHub repository](https://github.com/PyAV-Org/PyAV).
- Decord: Video processing library. Learn more on [GitHub](https://github.com/dmlc/decord).

**NVIDIA CuDNN**
- GPU-accelerated library for deep neural networks. Download and optimize workflows with [NVIDIA CuDNN](https://developer.nvidia.com/cudnn).

**Torch and Torchvision**
- PyTorch: Open-source ML library. Get started at [official website](https://pytorch.org/).
- Torchvision: PyTorch's computer vision library. Explore documentation [here](https://pytorch.org/vision/stable/index.html).

In [ ]:
# Install the required packages
%cd /content/ict3104-team09-2023/
!pip install triton==2.0.0


#!pip install -q diffusers==0.11.1 transformers==4.26.0 bitsandbytes==0.35.4 imageio-ffmpeg xformers
!pip install -q diffusers==0.11.1 transformers==4.26.0 bitsandbytes==0.41.1 imageio-ffmpeg #change bitsandbytes for mmpose

!pip install accelerate
!pip install omegaconf
!pip install einops
!pip install av
!pip install decord

!pip install -r requirements.txt

!pip install  xformers==0.0.19

!pip uninstall nvidia-cudnn-cu11 -y
!pip install nvidia-cudnn-cu11==8.6.0.163


!pip uninstall torch torchvision -y
#!pip install torch torchvision -f https://download.pytorch.org/whl/cu111/torch_stable.html
!pip install torch==2.0.0 torchvision -f https://download.pytorch.org/whl/cu118/torch_stable.html #change to cuda 11.8 for mmpose



/content/ict3104-team09-2023
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 19.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 861.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

**Step 3 - Create checkpoints directory and clone into project**

The checkpoint file will be too large to be included in our github project repo, thus we clone the checkpoint separately.

In [ ]:
# Create checkpoints directory and clone another repository
%cd /content/ict3104-team09-2023/
%mkdir checkpoints
!git lfs install
!git clone https://huggingface.co/YueMafighting/FollowYourPose_v1
!mv /content/ict3104-team09-2023/FollowYourPose_v1/* /content/ict3104-team09-2023/checkpoints/
!rm -rf FollowYourPose_v1

/content/ict3104-team09-2023
Updated git hooks.
Git LFS initialized.
Cloning into 'FollowYourPose_v1'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), 584.62 KiB | 2.15 MiB/s, done.
Filtering content: 100% (9/9), 9.75 GiB | 44.53 MiB/s, done.


# Data Exploration Section

**Step 4 - This section allows you to view a playback of the charades videos.**

1. After loading the cell, click on the dropdown list
2. The video playback of the chosen video file will be shown in the output cell

The videos included in the selection can be found at this path: "/content/ict3104-team09-2023/Data/CharadesVideos"

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Video

# Function to display the selected video
def display_video(video_name):
    clear_output(wait=True)  # Clear previous output, but retain the dropdown
    video_path = os.path.join(folder_path, video_name)
    # Use IPython's Video to display the video
    display(Video(video_path, embed=True))

# Path to the folder containing your videos
folder_path = '/content/ict3104-team09-2023/Data/CharadesVideos'  # Adjust the path accordingly

# List all video files in the folder and its subfolders
video_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(('.mp4', '.avi', '.mov', 'mov')):  # Add other video formats if needed
            video_files.append(os.path.relpath(os.path.join(root, file), start=folder_path))

# Create a dropdown widget for video selection
dropdown = widgets.Dropdown(
    options=video_files,
    description='Select Video:',
    disabled=False,
)

# Output area to display the video
output_area = widgets.Output()

# Display the dropdown and set up its event
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output_area:
            display_video(change['new'])

dropdown.observe(on_dropdown_change)
display(widgets.VBox([dropdown, output_area]))  # Display the dropdown and output area in a VBox


# Video Cropping Section

**Step 5 - Steps to perform video cropping**
1. After loading the cell, click on the dropdown list
2. Select the desired subfolder from the main folder
3. Press on the Crop Videos button, the videos in the subfolder will be cropped to 15 seconds, with the size of 512x512 pixels
4. The newly cropped videos will be saved back into the selected subfolders with the same naming + "1" at the back.

Main folder path :  '/content/ict3104-team09-2023/Data/CharadesVideos'

The purpose of cropping the video is to use for inference, training and MMPose.

Note: If you prefer not to overwrite it, please refrain from using the same video.

In [ ]:
import os
import moviepy.editor as mp
import ipywidgets as widgets
from IPython.display import display, HTML

# Function to process, crop, and save the video
def process_video(video_path, output_path):
    # Load the video and crop to 15 seconds
    clip = mp.VideoFileClip(video_path)
    clip = clip.subclip(0, 15)

    # Center crop the video to a square shape
    min_side = min(clip.w, clip.h)
    x_center = (clip.w - min_side) / 2
    y_center = (clip.h - min_side) / 2
    clip_cropped = clip.crop(x_center, y_center, x_center + min_side, y_center + min_side)

    # Resize the video to 512x512 pixels
    clip_resized = clip_cropped.resize(height=512, width=512)

    # Save the processed video
    clip_resized.write_videofile(output_path, codec='libx264')

# Function to handle processing when a folder is selected
def process_selected_folder(change):
    global selected_folder_path
    selected_folder_path = change['new']
    with output_area:
        output_area.clear_output()
        display(HTML(f"Selected folder: {selected_folder_path}"))
        crop_button.disabled = False  # Enable the button when a folder is selected

# Function to handle cropping when the button is pressed
def crop_videos(button):
    with output_area:
        output_area.clear_output()
        display(HTML("Processing videos..."))

        # Get the list of subfolders in the selected folder
        subfolders = [f for f in os.listdir(selected_folder_path) if os.path.isdir(os.path.join(selected_folder_path, f))]

        # Process each subfolder and its videos
        for subfolder in subfolders:
            subfolder_path = os.path.join(selected_folder_path, subfolder)

            # Get the list of video files in the subfolder
            video_files = [os.path.join(subfolder_path, file) for file in os.listdir(subfolder_path) if file.lower().endswith(('.mp4', '.avi', '.mov'))]

            # Process each video and save it back to its original folder
            for video_path in video_files:
                base_name, extension = os.path.splitext(os.path.basename(video_path))
                output_file = os.path.join(subfolder_path, f"{base_name}1{extension}")
                process_video(video_path, output_file)

                display(HTML(f"Processed video saved as: {os.path.basename(output_file)}"))

# Path to the main folder containing subfolders with videos
main_folder_path = '/content/ict3104-team09-2023/Data'

# Create a dropdown widget for folder selection
folder_dropdown = widgets.Dropdown(
    options=[os.path.join(main_folder_path, d) for d in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, d))],
    description='Select Folder:',
    disabled=False,
)

# Bind the folder selection to the processing function
folder_dropdown.observe(process_selected_folder, names='value')

# Create a button to trigger cropping
crop_button = widgets.Button(
    description="Crop Videos",
    disabled=True  # Disable initially until folder is selected
)
crop_button.on_click(crop_videos)

# Output area to display processing status
output_area = widgets.Output()

# Display the folder dropdown, crop button, and output area in a VBox
display(widgets.VBox([folder_dropdown, crop_button, output_area]))


# MMPose Generation Section

**Step 6 - Create mmpose directory, install required package and input MMPose**

Perform step **6a** to **6c** before proceeding to **step 7**.

**Step 6a** - Install the neceassry dependencies for the MMPose generation

In [ ]:
# create mmpose directory and install the required package
%mkdir /content/mmpose
%cd /content/mmpose

# install virtual environment
!pip install virtualenv
# create virtual environment
!virtualenv myenv

# install required package
!pip install openmim
!mim install mmengine
!mim install "mmcv>=2.0.1"
!mim install "mmdet>=3.1.0"

/content/mmpose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 25.6 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.0.0
    Uninstalling platformdirs-4.0.0:
      Successfully uninstalled platformdirs-4.0.0
created virtual environment CPython3.10.12.final.0-64 in 846ms
  creator CPython3Posix(dest=/content/mmpose/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==68.2.2, wheel==0.41.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 988.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.2 MB/s eta 0:00:00
    

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.1/450.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 30.1 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.4 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00


**Step 6b** - clone the MMPose repo into our project so that we can use the libraries.

In [ ]:
# git clone mmpose into the directory
!git clone https://github.com/open-mmlab/mmpose.git

# install requirement.txt file
%cd mmpose
!pip install -e .

Cloning into 'mmpose'...
remote: Enumerating objects: 30322, done.
remote: Counting objects: 100% (1098/1098), done.
remote: Compressing objects: 100% (562/562), done.
remote: Total 30322 (delta 571), reused 947 (delta 525), pack-reused 29224
Receiving objects: 100% (30322/30322), 53.59 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (21379/21379), done.
/content/mmpose/mmpose
Obtaining file:///content/mmpose/mmpose
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/jin-s13/xtcocoapi to /tmp/pip-install-k4sd3dg1/xtcocotools_185df0924fed46e193847c1351272855
  Running command git clone --filter=blob:none --quiet https://github.com/jin-s13/xtcocoapi /tmp/pip-install-k4sd3dg1/xtcocotools_185df0924fed46e193847c1351272855
  Resolved https://github.com/jin-s13/xtcocoapi to commit d74033ff1635e9002133b2380862bc2b728584d2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

**Step 6c - to run the input mmpose**
1. Run the cell.
2. Using the UI to select the input video path that you want to convert to skeleton.
3. Click run cell to obtain the skeleton video output.


Input Video Path:  '/content/ict3104-team09-2023/Data'

Output Video Path: '/content/ict3104-team09-2023/pose_example'

Expected Output: MP4 videos

In [ ]:
# generate mmpose skeleton video with UI
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Path to the main folder containing subfolders with videos
main_folder_path = '/content/ict3104-team09-2023/Data'

# Create a dropdown widget for folder selection
folder_dropdown = widgets.Dropdown(
    options=[os.path.join(main_folder_path, d) for d in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, d))],
    description='Select Folder:',
    disabled=False,
)

# Create a dropdown widget for subfolder selection
subfolder_dropdown = widgets.Dropdown(
    options=[],  # Options will be populated based on the selected folder
    description='Select Subfolder:',
    disabled=False,
)

# Create a dropdown widget for video selection
video_dropdown = widgets.Dropdown(
    options=[],  # Options will be populated based on the selected folder and subfolder
    description='Select Video:',
    disabled=False,
)

# Create a button for triggering skeleton generation
generate_button = widgets.Button(description="Generate Skeleton")

# Output widget for displaying messages
output = widgets.Output()

# Function to update subfolder options based on selected folder
def update_subfolder_options(change):
    selected_folder = change.new
    subfolders = [d for d in os.listdir(selected_folder) if os.path.isdir(os.path.join(selected_folder, d))]
    subfolder_dropdown.options = subfolders

folder_dropdown.observe(update_subfolder_options, names='value')

# Function to update video options based on selected folder and subfolder
def update_video_options(change):
    selected_folder = folder_dropdown.value
    selected_subfolder = subfolder_dropdown.value
    video_files = [f for f in os.listdir(os.path.join(selected_folder, selected_subfolder)) if f.endswith('.mp4')]
    video_dropdown.options = video_files

subfolder_dropdown.observe(update_video_options, names='value')

# Function to generate skeleton when the button is clicked
def generate_skeleton(_):
    input_folder = folder_dropdown.value
    selected_subfolder = subfolder_dropdown.value
    selected_video = video_dropdown.value
    input_path = os.path.join(input_folder, selected_subfolder, selected_video)
    output_path = '/content/ict3104-team09-2023/pose_example'

    with output:
        clear_output(wait=True)
        print(f"Generating skeleton for {selected_video}...")
        !python /content/mmpose/mmpose/demo/inferencer_demo.py {input_path} \
            --pose2d 'human'  \
            --vis-out-dir {output_path} \
            --thickness '5'\
            --black-background
        print("Skeleton generation completed!")

generate_button.on_click(generate_skeleton)

# Display the widgets
display(folder_dropdown)
display(subfolder_dropdown)
display(video_dropdown)
display(generate_button)
display(output)

FileNotFoundError: ignored

# Inference Section

**Step 7 - This section allows you to choose an input video from the subfolder, and write in a text prompt as input into the genAI model.**

1. After loading the code, select the desired pre-trained model from the dropdown list and choose a video
2. Enter the prompts
3. Press on the Run button

Perform **Step** **7a** to **7c** before proceed to** step 8**

**Step 7a** - Install dependencies

In [ ]:
!pip install asyncio

**Step 7b** - Select the inference config, videos, input prompts and video length,then pressed the run button. This will modify the config file and run the inference command.

Output Result Path:

'/content/ict3104-team09-2023/checkpoints/inference/sample-xxx'

xxx will determine by the date and time you perform the run.

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import torch
import yaml
import subprocess
import asyncio

%cd /content/ict3104-team09-2023/

# Load pre-trained genAI model
# Config file path
config_path = '/content/ict3104-team09-2023/configs/'

# List all config files in the folder
config_files = [f for f in os.listdir(config_path) if f.endswith(('.yaml'))]

# Create a dropdown widget for loading pre-trained model
dropdown_pretrained = widgets.Dropdown(
    options=config_files,
    description='Pre-trained:',
    disabled=False,
)

display(widgets.VBox([dropdown_pretrained]))

# Choose videos from the data folder and create input text prompt
# Choose input videos (skeleton_path)
folder_path = '/content/ict3104-team09-2023/pose_example/'

# List all video files in the folder
video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov', 'MOV'))]

# Create a dropdown widget for video selection (allowing multiple selection)
dropdown = widgets.SelectMultiple(
    options=video_files,
    description='Select Videos:',
    disabled=False,
)

display(widgets.VBox([dropdown]))

# Create textarea widget for text prompts
text_prompt = widgets.Textarea(
    description='Prompts:',
    disabled=False,
    layout=widgets.Layout(width='80%', height='100px')
)

def modify_prompt(change):
    # Modify the config file
    selected_file = os.path.join(config_path, dropdown_pretrained.value)
    with open(selected_file, 'r') as yaml_file:
        config = yaml.safe_load(yaml_file)

    # Modify the specific value in the config file
    prompts = [line.strip() for line in change.new.split('\n') if line.strip()]
    config['validation_data']['prompts'] = prompts

    # Save config file after modification
    with open(selected_file, 'w') as yaml_file:
        yaml.dump(
            config,
            yaml_file,
            default_flow_style=False,
            indent=2,  # Use 2 spaces for indentation
            width=float("inf"),  # Disable line wrapping
            allow_unicode=True  # Allow non-ASCII characters
        )

text_prompt.observe(modify_prompt, names='value')
display(widgets.VBox([text_prompt]))

# Create textarea widget for modifying video length
video_length_input = widgets.Text(
    description='Video Length:',
    disabled=False,
)

def modify_video_length(change):
    # Modify the config file
    selected_file = os.path.join(config_path, dropdown_pretrained.value)
    with open(selected_file, 'r') as yaml_file:
        config = yaml.safe_load(yaml_file)

    # Check if the input is a valid integer
    try:
        video_length = int(change.new)
    except ValueError:
        # Handle the case where the input is not a valid integer
        print("Invalid input for video length. Please enter a valid integer.")
        return

    # Modify the specific value in the config file
    config['validation_data']['video_length'] = video_length

    # Save config file after modification
    with open(selected_file, 'w') as yaml_file:
        yaml.dump(
            config,
            yaml_file,
            default_flow_style=False,
            indent=2,  # Use 2 spaces for indentation
            width=float("inf"),  # Disable line wrapping
            allow_unicode=True  # Allow non-ASCII characters
        )

# Use on_submit event instead of observe
video_length_input.observe(modify_video_length, names='value')
display(widgets.VBox([video_length_input]))

# Create progress bar widget
progress_bar = widgets.FloatProgress(
    value=0.0,
    min=0.0,
    max=1.0,
    description='Progress:',
    bar_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'blue'},  # Optional custom color for the progress bar
    layout=widgets.Layout(width='80%')
)

# Display the progress bar
display(widgets.VBox([progress_bar]))

# Create button to run txt2video sequentially for each selected video
async def run_txt2video(_):
    config_file = f"/content/ict3104-team09-2023/configs/{dropdown_pretrained.value}"
    selected_videos = [f"/content/ict3104-team09-2023/pose_example/{video}" for video in dropdown.value]

    # Set the total number of steps based on the number of selected videos
    total_steps = len(selected_videos)

    for i, video_path in enumerate(selected_videos, start=1):
        # Update progress bar value
        progress_bar.value = i / total_steps

        # Build the command
        command = f"TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch /content/ict3104-team09-2023/txt2video.py --config={config_file} --skeleton_path={video_path}"

        print("Executing command:", command)

        # Run the command and capture the output using await
        result = await asyncio.to_thread(subprocess.run, command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Print the stdout and stderr
        display(Markdown(f"**Command Output for {video_path}:**\n```\n{result.stdout}\n{result.stderr}\n```"))

    # Reset progress bar after completion
    progress_bar.value = 0.0

run_button = widgets.Button(description="Run txt2video for selected videos")
run_button.on_click(lambda x: asyncio.create_task(run_txt2video(x)))  # Use asyncio.create_task to run the async function
display(widgets.VBox([run_button]))


/content/ict3104-team09-2023


Executing command: TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch /content/ict3104-team09-2023/txt2video.py --config=/content/ict3104-team09-2023/configs/inference_config.yaml --skeleton_path=/content/ict3104-team09-2023/pose_example/vis_ikun_pose2.mov


**Step 7c** - This section allows you to select an charades / original input videos and the output gif to be display side by side for obervations.

Input videos :  '/content/ict3104-team09-2023/Data/CharadesVideos'

output videos : '/content/ict3104-team09-2023/checkpoints/inference'

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, Video, Image, HTML
import ipywidgets as widgets


# Function to display the selected video with inline resizing
def display_video(video_name, folder_path, width):
    clear_output(wait=True)
    video_path = os.path.join(folder_path, video_name)
    display(HTML(f"<b>{video_name}</b><br>"))
    display(Video(video_path, embed=True, width=width))

def display_all_inference_gifs(folder_path, width):
    gif_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.gif')]

    gif_widgets = []  # Create a list to hold the widget for each GIF
    with output_area_gif_2:
        for gif_file in gif_files:
            gif_path = os.path.join(folder_path, gif_file)
            gif_name = os.path.splitext(gif_file)[0]  # Get the name of the GIF without extension

            # Create a label widget for the GIF name
            label = widgets.Label(value=gif_name)

            # Create an image widget for the GIF
            img2 = open(gif_path, 'rb').read()
            wi = widgets.Image(value=img2, format='gif', width=width)

            # Create a VBox to display the label and GIF widget vertically
            vbox = widgets.VBox([label, wi])
            gif_widgets.append(vbox)

        # Create an HBox to display the GIF widgets side by side
        gif_display = widgets.HBox(gif_widgets)
        display(gif_display)


# Path to the first folder containing your videos
folder_path = '/content/ict3104-team09-2023/Data/CharadesVideos'  # Adjust the path accordingly

# List all video files in the first folder and its subfolders
video_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(('.mp4', '.avi', '.mov', 'mov')):  # Add other video formats if needed
            video_files.append(os.path.relpath(os.path.join(root, file), start=folder_path))

# Create a dropdown widget for video selection in the first folder
dropdown = widgets.Dropdown(
    options=video_files,
    description='Input Video:',
    disabled=False,
)

# Output area to display the video from the first folder
output_area = widgets.Output()

# Display the first dropdown and set up its event
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output_area:
            display_video(change['new'], folder_path, width=300)  # Adjust the width as needed

dropdown.observe(on_dropdown_change)

# Path to the second folder containing your inference GIF folders
folder_path_2 = '/content/ict3104-team09-2023/checkpoints/inference'  # Adjust the path accordingly

# List all subfolders in the second folder
inference_folders = [d for d in os.listdir(folder_path_2) if os.path.isdir(os.path.join(folder_path_2, d))]

# Create a dropdown widget for selecting an inference folder
dropdown_inference_folder = widgets.Dropdown(
    options=inference_folders,
    description='Inference Output:',
    disabled=False,
)

# Output area to display the GIFs from the selected folder
output_area_gif_2 = widgets.Output()

# Display the selected folder's GIFs when a new folder is selected
def on_dropdown_change_inference_folder(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_folder = change['new']
        folder_path_selected = os.path.join(folder_path_2, selected_folder)
        with output_area_gif_2:
            clear_output(wait=True)
            display_all_inference_gifs(folder_path_selected, width=300)  # Adjust the width as needed

dropdown_inference_folder.observe(on_dropdown_change_inference_folder)

# Create an HBox layout to display the two dropdowns side by side
video_display = widgets.HBox([output_area, output_area_gif_2])
display(widgets.VBox([dropdown, dropdown_inference_folder, video_display]))

# Superimposed mmpose

The purpose of superimpose is to compare the skeletion obtained from MMPose and the charades / original  video and the outputted gif.

**Step 8:**
1.   Overlay the skeleton on input video
2.   Overlay the skeleton on output gif

A video that contains the skeleton generated by MMPose superimposed onto the resulting video will be saved in the /content/ict3104-team09-2023/pose_example/ directory.

**Step8a - MMPOSE for gif**

*   Run the cell to install required package
*   Run the cell to convert outputted gif to a mp4 format
*   Generate the skeleton video for the outputted gif

Note: modify the input path : '/content/ict3104-team09-2023/checkpoints/inference/sample-xxx''

In [ ]:
pip install imageio[ffmpeg]

In [ ]:
import os
import imageio

# Input folder containing GIFs
input_folder = '/content/ict3104-team09-2023/checkpoints/inference/sample-1000-33-2023-11-19T16-40-40'

# Output folder for MP4 files
output_folder = '/content/ict3104-team09-2023/checkpoints/inference/mp4_output/'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Set the desired FPS for the output MP4
fps = 10

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.gif'):
        # Remove spaces from the filename
        filename_no_spaces = filename.replace(' ', '_')

        input_gif = os.path.join(input_folder, filename)
        output_mp4 = os.path.join(output_folder, f'{os.path.splitext(filename_no_spaces)[0]}.mp4')  # Use filename_no_spaces

        # Read the GIF and save it as an MP4
        with imageio.get_reader(input_gif) as reader, imageio.get_writer(output_mp4, fps=fps) as writer:
            for frame in reader:
                writer.append_data(frame)

print("Conversion of GIFs to MP4 completed.")


In [ ]:
# generate mmpose skeleton video for gif
import os

input_folder = '/content/ict3104-team09-2023/checkpoints/inference/mp4_output'
output_folder = '/content/ict3104-team09-2023/pose_example'

# List all MP4 files in the input folder
mp4_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]

# Loop through each MP4 file and generate skeletons
for mp4_file in mp4_files:
    input_path = os.path.join(input_folder, mp4_file)
    output_path = os.path.join(output_folder, mp4_file.replace('.mp4', '_skeleton.mp4'))

    # Generate skeleton using the provided command
    command = f"python /content/mmpose/mmpose/demo/inferencer_demo.py {input_path} \
        --pose2d 'human'  \
        --vis-out-dir {output_folder} \
        --thickness '5'\
        --black-background"

    os.system(command)

    print(f"Skeleton video generated for {mp4_file}. Output saved to {output_path}")


**Step 8b - Overlay skeleton on input video**

**Note: Before you run the cell, please modify the below paths to select your wanted input video, skeleton and name the output file**

Input video path: "/content/ict3104-team09-2023/Data/CharadesVideos/TW3RS/TW3RS1.mp4" # input video, example : TW3RS/TW3RS1.mp4

Generated Skeleton path: "/content/ict3104-team09-2023/pose_example/TW3RS1.mp4" # input skeleton exmaple: TW3RS1.mp4

Output path: "/content/ict3104-team09-2023/pose_example/overlay_skeleton_TW3RS1.mp4" # combine/overlay of skeleton and inputvideo, exmaple :overlay_skeleton_TW3RS1.mp4"


In [ ]:
import cv2
import os
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

# overlay skeleton on input video
def preprocess_frames(frames):
    preprocessed_frames = []
    for frame in frames:
        # Preprocess each frame
        frame = cv2.resize(frame, (299, 299))  # Resize frame to (299, 299)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frame = np.expand_dims(frame, axis=0)  # Add batch dimension
        frame = preprocess_input(frame)  # Preprocess for InceptionV3
        preprocessed_frames.append(frame)
    return np.vstack(preprocessed_frames)

# Indicate the path for real and generated video
real_video_path = "/content/ict3104-team09-2023/Data/CharadesVideos/TW3RS/TW3RS1.mp4" # input video
generated_video_path = "/content/ict3104-team09-2023/pose_example/TW3RS1.mp4" # input skeleton

# Output video path for video overlay
output_video_path = "/content/ict3104-team09-2023/pose_example/overlay_skeleton_TW3RS1.mp4"

# Read frames from videos
real_frames = []
generated_frames = []

cap_real = cv2.VideoCapture(real_video_path)
cap_generated = cv2.VideoCapture(generated_video_path)

# Get the frame width, height, and frame rate of the generated video
frame_width = int(cap_generated.get(3))
frame_height = int(cap_generated.get(4))
frame_rate = int(cap_generated.get(5))

# Define the codec for the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Create the output video writer
output_video = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height))

while True:
    ret, frame_real = cap_real.read()
    ret, frame_generated = cap_generated.read()

    if not ret:
        break

    real_frames.append(frame_real)
    generated_frames.append(frame_generated)

    # Resize the skeleton frame to match the dimensions of the generated video frame
    frame_generated = cv2.resize(frame_generated, (frame_width, frame_height))

    # Overlay the skeleton frame onto the generated frame
    combined_frame = cv2.addWeighted(frame_real, 1, frame_generated, 0.5, 0)

    # Write the combined frame to the output video
    output_video.write(combined_frame)

# Preprocess frames
real_frames_preprocessed = preprocess_frames(real_frames)
generated_frames_preprocessed = preprocess_frames(generated_frames)

# Release video capture objects
cap_real.release()
cap_generated.release()
output_video.release()
print("Overlay generation completed!")


Overlay generation completed!


**Step 8c** - Overlay skeleton on Outputted gif


**For this step will be slightly different from step 8b, will need to do some preprocessing. First we will convert the skeletion video to gif.**

Note:

1) Please modify the code, change the input video path to select the mp4 video that you want.
Example: Iron_man_on_the_beach.mp4

2) Please modify the code, change the output gif path, you can name anything.

Example: skeleton_gif1.gif

In [ ]:
#Convert skeleton video to gif
import imageio

# Input MP4 filename
input_mp4 = '/content/ict3104-team09-2023/pose_example/Iron_man_on_the_beach/Iron_man_on_the_beach.mp4'

# Output GIF filename
output_gif = '/content/ict3104-team09-2023/pose_example/skeleton_gif1.gif'

# Read the MP4 and save it as a GIF
with imageio.get_reader(input_mp4) as reader, imageio.get_writer(output_gif, mode='I', duration=1/reader.get_meta_data()['fps']) as writer:
    for frame in reader:
        writer.append_data(frame)
print("Convert skeleton video to gif completed")

**After obtained the generated gif above, example Skeleton_gif1.gif**

Next we will perform overlay on the gif:

Input gif path: '/content/ict3104-team09-2023/checkpoints/inference/' # input gif

Generated Skeleton path: /content/ict3104-team09-2023/pose_example/ # input skeleton

Output path: '/content/ict3104-team09-2023/pose_example/overlay_gif_skeleton3.gif' # combine/overlay of skeleton and inputvideo

Note the outpath, the [overlay_gif_skeleton3.gif] can rename to anything you want to name as your output.


In [ ]:
import os
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import imageio

# overlay skeleton on output video
def overlay_skeleton(real_video_path, generated_video_path, output_gif_path):
    cap_real = cv2.VideoCapture(real_video_path)
    cap_generated = cv2.VideoCapture(generated_video_path)

    # Create a list to store frames of the output GIF
    output_frames = []

    # Iterate through the frames of the real and generated videos
    while True:
        ret_real, frame_real = cap_real.read()
        ret_generated, frame_generated = cap_generated.read()

        if not ret_real or not ret_generated:
            break

        # Ensure both frames have the same dimensions
        if frame_real.shape != frame_generated.shape:
            raise ValueError("Video frames must have the same dimensions.")

        # Overlay the skeleton frame onto the real frame
        alpha = 0.5  # Adjust the alpha value for the overlay
        combined_frame = cv2.addWeighted(frame_real, 1 - alpha, frame_generated, alpha, 0)

        # Append the combined frame to the output frames list
        output_frames.append(combined_frame)

    # Write the output frames to a new GIF
    output_gif = imageio.get_writer(output_gif_path, mode='I')

    for frame in output_frames:
        output_gif.append_data(frame)

    output_gif.close()

# Function to get all MP4 files recursively in a directory
def get_all_gif_files(directory):
    gif_files = []
    for root, dirs, files in os.walk(directory):
        gif_files += [os.path.join(root, file) for file in files if file.endswith('.gif')]
    return gif_files

# UI for input paths and overlaying GIF
real_gif_folder_text = widgets.Text(
    value='/content/ict3104-team09-2023/checkpoints/inference/',
    description='Real Gif Folder:',
    disabled=False
)
display(real_gif_folder_text)

generated_gif_folder_text = widgets.Text(
    value='/content/ict3104-team09-2023/pose_example/',
    description='Generated Gif Folder:',
    disabled=False
)
display(generated_video_folder_text)

real_gif_dropdown = widgets.Dropdown(
    options=get_all_gif_files(real_gif_folder_text.value),
    value=None,
    description='Choose Real Gif:',
    disabled=False,
)
display(real_gif_dropdown)

generated_gif_dropdown = widgets.Dropdown(
    options=get_all_gif_files(generated_gif_folder_text.value),
    value=None,
    description='Choose Generated Gif:',
    disabled=False,
)
display(generated_gif_dropdown)

output_gif_path_text = widgets.Text(
    value='/content/ict3104-team09-2023/pose_example/overlay_gif_skeleton3.gif',
    description='Output GIF Path:',
    disabled=False
)
display(output_gif_path_text)

output = widgets.Output()

def overlay(_):
    with output:
        clear_output(wait=True)
        real_video_path = real_gif_dropdown.value
        generated_video_path = generated_gif_dropdown.value
        output_gif_path = output_gif_path_text.value

        overlay_skeleton(real_video_path, generated_video_path, output_gif_path)

        # Display the overlayed GIF
        print("Overlay output gif completed")


overlay_button = widgets.Button(description="Overlay")
overlay_button.on_click(overlay)
display(overlay_button)
display(output)

# FID (Fretchet Inception Distance)

FID is a metric to evaluating quality of generated images and performance of genAI generation. The lower the number the better, better as in evaluate the accuracy of generated input video and output gif.


**Step 9**:
1. Before running the cell, please manually change the input video and output video path accordingly. (Please select the same input video you and it's respective output sample folder)
2. Run the cell
3. Select the gif that you want to calculate fid
4. Click "calculate FID"


Input video path: /content/ict3104-team09-2023/pose_example/TVQO91.mp4',

Output video path: '/content/ict3104-team09-2023/checkpoints/inference/sample-1000-33-2023-11-18T08-54-50/'

Note: [TVQO91.mp4], [sample-1000-33-2023-11-18T08-54-50] varies

In [ ]:
import os
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.linalg import sqrtm

# Load pre-trained InceptionV3 model without top classification layers
inception_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')

def preprocess_frames(frames):
    preprocessed_frames = []
    for frame in frames:
        # Preprocess each frame
        frame = cv2.resize(frame, (299, 299))  # Resize frame to (299, 299)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frame = np.expand_dims(frame, axis=0)  # Add batch dimension
        frame = preprocess_input(frame)  # Preprocess for InceptionV3
        preprocessed_frames.append(frame)
    return np.vstack(preprocessed_frames)

def calculate_fid(real_frames, generated_frames):
    # Get InceptionV3 feature representations for real and generated frames
    real_features = inception_model.predict(real_frames)
    generated_features = inception_model.predict(generated_frames)

    # Calculate mean and covariance statistics for real and generated features
    mu_real, sigma_real = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu_generated, sigma_generated = np.mean(generated_features, axis=0), np.cov(generated_features, rowvar=False)

    # Calculate Frechet Distance between real and generated distributions
    sqrt_term = sqrtm(np.dot(sigma_real, sigma_generated))
    if np.iscomplexobj(sqrt_term):
        sqrt_term = sqrt_term.real
    fid = np.linalg.norm(mu_real - mu_generated) + np.trace(sigma_real + sigma_generated - 2 * sqrt_term)
    return fid

# UI for input paths and choosing GIF
real_video_path_text = widgets.Text(
    value='/content/ict3104-team09-2023/pose_example/TVQO91.mp4',
    description='Real Video Path:',
    disabled=False
)
display(real_video_path_text)

generated_video_folder_text = widgets.Text(
    value='/content/ict3104-team09-2023/checkpoints/inference/sample-1000-33-2023-11-18T08-54-50/',
    description='Generated Video Folder:',
    disabled=False
)
display(generated_video_folder_text)

gif_dropdown = widgets.Dropdown(
    options=[f for f in os.listdir(generated_video_folder_text.value) if f.endswith('.gif')],
    value=None,
    description='Choose GIF:',
    disabled=False,
)
display(gif_dropdown)

output = widgets.Output()

def calculate_fid_and_display(_):
    with output:
        clear_output(wait=True)
        real_video_path = real_video_path_text.value
        generated_video_path = os.path.join(generated_video_folder_text.value, gif_dropdown.value)

        # Read frames from videos
        real_frames = []
        generated_frames = []
        cap_real = cv2.VideoCapture(real_video_path)
        cap_generated = cv2.VideoCapture(generated_video_path)

        # Determine the minimum number of frames
        min_frames = min(int(cap_real.get(cv2.CAP_PROP_FRAME_COUNT)), int(cap_generated.get(cv2.CAP_PROP_FRAME_COUNT)))
        for _ in range(min_frames):
            ret, frame_real = cap_real.read()
            ret, frame_generated = cap_generated.read()
            if not ret:
                break
            real_frames.append(frame_real)
            generated_frames.append(frame_generated)

        # Check if the frames lists are not empty
        if not real_frames or not generated_frames:
            print(f"Error: Frames not read properly from {real_video_path} or {generated_video_path}.")
        else:
            # Preprocess frames
            real_frames_preprocessed = preprocess_frames(real_frames)
            generated_frames_preprocessed = preprocess_frames(generated_frames)

            # Calculate FID score
            fid_score = calculate_fid(real_frames_preprocessed, generated_frames_preprocessed)

            # Check if FID score is valid (not too large)
            if np.isnan(fid_score) or np.isinf(fid_score):
                print(f"Error: Invalid FID score for {generated_video_path}.")
            else:
                print(f"FID Score for {generated_video_path}: {fid_score}")

        # Release video capture objects
        cap_real.release()
        cap_generated.release()

calculate_button = widgets.Button(description="Calculate FID")
calculate_button.on_click(calculate_fid_and_display)
display(calculate_button)
display(output)

1/1 [==============================] - 0s 43ms/step
FID Score for Iron man on the street.gif: 1.0662847283317183e+100


# Training Section

**Step 10 - to Train a model**
1. After loading the cell, enter all the config information for training the model and select a video

**Informations field:**

**set config file name** - is to give the config file a name.

**Output path** - is to set the path for the output generated from training (Is to prevent overwritting)

train video path - select which datasets used to trained

**train prompt** - what ever prompt you want to use to train/insert (only one prompt)

video length - length of the video

**max_train_steps** - number of steps you want to train the model (higher steps higher accuracy but also take longer)

**checkpointing** - which point of time you want to save the training.

2. Press on the Save Config button
3. Run the training command (Modify the config file name before running )

The config file will be saved under /content/ict3104-team09-2023/configs/

**After train finised, an output file will be generated, to use the generate output file. Inside the output file, there will be a checkpoint-50 folder, to use this folder refer to the inference_config.yml file. Modify the resume_from_checkpoint: ./checkpoints/followyourpose_checkpoint-1000 to use it.**

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display
from omegaconf import OmegaConf

%cd /content/ict3104-team09-2023/

# Model path of the initial model
MODEL_PATH = '/content/ict3104-team09-2023/configs'

# Path to the folder containing your videos
data_path = '/content/ict3104-team09-2023/Data/'

# List all folders in the path
folders = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]

# Create widgets for train config parameters
config_name_widget = widgets.Text(value="filename.yaml", description="Config Name:")
train_video_path_widget = widgets.Dropdown(options=folders, description="Train Video Path:")
train_prompt_widget = widgets.Text(value="a man is skiing", description="Train Prompt:")
video_length_widget = widgets.IntText(value=8, description="Video Length(sec):")
max_train_steps_widget = widgets.IntText(value=50, description="Max Training Step:")
checkpointing_widget = widgets.IntText(value=25, description="Save Checkpoint every ? Step:")
output_widget = widgets.Text(value="output/model-1", description="Output Path:")

# Function to save the configuration
def save_config(button):
    output_path = output_widget.value
    config = {
        "pretrained_model_path": "./checkpoints/stable-diffusion-v1-4",
        "output_dir": output_path,
        "train_data": {
            "video_path": os.path.join(data_path, train_video_path_widget.value),
            "prompt": train_prompt_widget.value,
            "n_sample_frames": 4,
            "width": 512,
            "height": 512,
            "sample_start_idx": 0,
            "sample_frame_rate": 2,
            "dataset_set": "train"
        },
        "validation_data": {
            "prompts": [
                "A Iron man on the beach",
                "A Spider man on the snow",
                "A Superman on the street",
                "A boy on the forest"
            ],
            "video_length": video_length_widget.value,
            "width": 256,
            "height": 256,
            "num_inference_steps": 50,
            "guidance_scale": 12.5,
            "use_inv_latent": False,
            "num_inv_steps": 50,
            "dataset_set": "val"
        },
        "learning_rate": 3e-5,
        "train_batch_size": 1,
        "max_train_steps": max_train_steps_widget.value,
        "checkpointing_steps":checkpointing_widget.value,
        "validation_steps": 100,
        "trainable_modules": [
            "attn1.to_q",
            "attn2.to_q",
            "attn_temp",
            "conv_temporal"
        ],
        "skeleton_path": './pose_example/vis_kun_pose2.mov',
        "seed": 33,
        "mixed_precision": "no",
        "use_8bit_adam": False,
        "gradient_checkpointing": False,
        "enable_xformers_memory_efficient_attention": True #True
    }

    # Get the full file path by joining the MODEL_PATH and the YAML file name
    file_path = os.path.join(MODEL_PATH, config_name_widget.value)

    with open(file_path, 'w') as config_file:
        config_file.write(OmegaConf.to_yaml(config))

    print(f"Config saved to: {file_path}")

# Create a button to save the configuration
save_button = widgets.Button(description="Save Config")
save_button.on_click(save_config)

# Display widgets and save button
widgets_container = widgets.VBox([
    config_name_widget,
    output_widget,
    train_video_path_widget,
    train_prompt_widget,
    video_length_widget,
    max_train_steps_widget,
    checkpointing_widget,
    save_button
])

display(widgets_container)


/content/ict3104-team09-2023


In [ ]:
! TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch train_followyourpose.py --config="configs/test_train.yaml"

2023-10-28 08:57:00.547794: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-28 08:57:00.547852: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-28 08:57:00.547895: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-28 08:57:00.559269: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-28 08:57:02.394027: W tensorflow/c